# Welcome to conu demo! 

You need to install following packages to be able to run docker examples: `docker` and `conu`, for kubernetes and OpenShift examples I recommend to run local cluster using `minishift`.

# Docker example

In [1]:
from conu import DockerBackend

In [2]:
backend = DockerBackend()

11:20:30.912 backend.py        INFO   conu has initiated, welcome to the party!
11:20:31.965 __init__.py       INFO   docker environment info: 'Client: Docker Engine - Community\n Version:           18.09.1\n API version:       1.39\n Go version:        go1.10.6\n Git commit:        4c52b90\n Built:             Wed Jan  9 19:33:12 2019\n OS/Arch:           darwin/amd64\n Experimental:      false\n\nServer: Docker Engine - Community\n Engine:\n  Version:          18.09.1\n  API version:      1.39 (minimum version 1.12)\n  Go version:       go1.10.6\n  Git commit:       4c52b90\n  Built:            Wed Jan  9 19:41:49 2019\n  OS/Arch:          linux/amd64\n  Experimental:     true\n'


In [4]:
image = backend.ImageClass("docker.io/library/nginx")

In [5]:
image.inspect()

{'Id': 'sha256:06144b2878448774e55577ae7d66b5f43a87c2e44322b3884e4e6c70d070b262',
 'RepoTags': ['nginx:latest'],
 'RepoDigests': ['nginx@sha256:24a0c4b4a4c0eb97a1aabb8e29f18e917d05abfe1b7a7c07857230879ce7d3d3'],
 'Parent': '',
 'Comment': '',
 'Created': '2018-09-05T00:57:00.322491744Z',
 'Container': 'ea0a37cdc91d1228880c77a4660fd74a6ba6ae5d2de6c54e2ac025c66c11f257',
 'ContainerConfig': {'Hostname': 'ea0a37cdc91d',
  'Domainname': '',
  'User': '',
  'AttachStdin': False,
  'AttachStdout': False,
  'AttachStderr': False,
  'ExposedPorts': {'80/tcp': {}},
  'Tty': False,
  'OpenStdin': False,
  'StdinOnce': False,
  'Env': ['PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
   'NGINX_VERSION=1.15.3-1~stretch',
   'NJS_VERSION=1.15.3.0.2.3-1~stretch'],
  'Cmd': ['/bin/sh', '-c', '#(nop) ', 'CMD ["nginx" "-g" "daemon off;"]'],
  'ArgsEscaped': True,
  'Image': 'sha256:674565b18e1c8e0102c657d89059818ba8e0f476073f63078439951d4e0370b0',
  'Volumes': None,
  'WorkingDir': '

In [ ]:
container = image.run_via_binary()

In [ ]:
container.is_running()

In [ ]:
container.get_IPv4s()

In [ ]:
container.get_ports()

In [ ]:
container.stop()

In [ ]:
container.is_running()

### HTTP client

In [6]:
image = backend.ImageClass("registry.fedoraproject.org/fedora")

11:27:20.753 image.py          INFO   Pulling from fedora
11:27:21.377 image.py          INFO   Pulling fs layer
11:27:22.257 image.py          INFO   Downloading
11:27:22.371 image.py          INFO   Downloading
11:27:22.499 image.py          INFO   Downloading
11:27:22.607 image.py          INFO   Downloading
11:27:22.722 image.py          INFO   Downloading
11:27:22.843 image.py          INFO   Downloading
11:27:23.199 image.py          INFO   Downloading
11:27:23.306 image.py          INFO   Downloading
11:27:23.423 image.py          INFO   Downloading
11:27:23.522 image.py          INFO   Downloading
11:27:23.645 image.py          INFO   Downloading
11:27:23.761 image.py          INFO   Downloading
11:27:23.863 image.py          INFO   Downloading
11:27:23.975 image.py          INFO   Downloading
11:27:24.269 image.py          INFO   Downloading
11:27:24.377 image.py          INFO   Downloading
11:27:24.499 image.py          INFO   Downloading
11:27:24.602 image.py          INFO  

In [7]:
container = image.run_via_binary(command=["python3", "-m", "http.server", "--bind", "0.0.0.0 8000"])

11:28:24.180 image.py          INFO   run container via binary in background


In [8]:
container.is_running()

True

In [9]:
request = container.http_request(port="8080")

ConnectionError: HTTPConnectionPool(host='172.17.0.2', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109732f60>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
request.ok

# Kubernetes example

This example demonstrating how to run image created by `DockerBackend` as container inside Kubernetes pod.

In [ ]:
from conu import K8sBackend

In [ ]:
k8s_backend = K8sBackend()

In [ ]:
docker_backend = DockerBackend()

In [ ]:
image = docker_backend.ImageClass("openshift/hello-openshift")

In [ ]:
pod = image.run_in_pod(namespace='myproject')

In [ ]:
pod.is_ready()

In [ ]:
pod.get_phase()

In [ ]:
pod.delete()

In [ ]:
pod.get_phase()

# OpenShift example

In [ ]:
from conu.utils import get_oc_api_token

In [ ]:
from conu import OpenshiftBackend

In [ ]:
api_key = get_oc_api_token()

In [ ]:
api_key

In [ ]:
openshift_backend = OpenshiftBackend(api_key=api_key)

In [ ]:
app_name = openshift_backend.deploy_image("centos/mariadb-102-centos7",
                                         oc_new_app_args=["--env", "MYSQL_ROOT_PASSWORD=test"],
                                         project='myproject')

In [ ]:
openshift_backend.all_pods_are_ready(app_name)

In [ ]:
openshift_backend.get_logs(app_name)

In [ ]:
openshift_backend.clean_project(app_name)

## Creating application from source

In [ ]:
openshift_backend.get_status()

In [ ]:
app_name = openshift_backend.create_new_app_from_source("centos/python-36-centos7",
                                                       source="https://github.com/openshift/django-ex.git",
                                                       project="myproject")

In [ ]:
openshift_backend.all_pods_are_ready(app_name)

In [ ]:
openshift_backend.get_status()

In [ ]:
openshift_backend.clean_project(app_name)

For more information see upstream GitHub repository: https://github.com/user-cont/conu. You can find there more examples for all supported backends.